## GooLeNet网络代码复现

### 导入python库

In [2]:
import torch
from torch import nn
from torchinfo import summary

1. Inception块

2. 辅助分类器

3. conv + BN + ReLu

### conv + BN + ReLu

In [3]:
class BasicCon2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs): # 选填参数 **kwargs
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, bias=False, **kwargs),
                                  nn.BatchNorm2d(out_channels),
                                  nn.ReLU(inplace=True))
    def forward(self, x):
        x = self.conv(x)
        return x
        

In [4]:
BasicCon2d(3,5,kernel_size=3)

BasicCon2d(
  (conv): Sequential(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
)

### Inception块
![picture](./Inception.jpg "picture")

In [5]:
class Inception(nn.Module):
    def __init__(self, 
                 in_channels, 
                 c1, 
                 c2, 
                 c3, 
                 c4):
        super().__init__()
        self.branch1 = BasicCon2d(in_channels, c1, kernel_size=1)
        self.branch2 = nn.Sequential(BasicCon2d(in_channels, c2[0], kernel_size=1),
                                BasicCon2d(c2[0], c2[1], kernel_size=3, padding=1))
        self.branch3 = nn.Sequential(BasicCon2d(in_channels, c3[0], kernel_size=1),
                                BasicCon2d(c3[0], c3[1], kernel_size=5, padding=2))
        self.branch4 = nn.Sequential(nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
                                BasicCon2d(in_channels, c4, kernel_size=1))


    def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        return torch.cat([b1, b2, b3, b4], dim=1)

In [6]:
Inception(192, 64, (96, 128), (16, 32), 32)

Inception(
  (branch1): BasicCon2d(
    (conv): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (branch2): Sequential(
    (0): BasicCon2d(
      (conv): Sequential(
        (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): BasicCon2d(
      (conv): Sequential(
        (0): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
  )
  (branch3): Sequential(
    (0): BasicCon2d(
      (conv): Sequential(
        (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): Batc

In [7]:
inception3a = Inception(192, 64, (96, 128), (16, 32), 32)

In [8]:
data = torch.ones(30,192,28,28)

In [9]:
inception3a(data).shape

torch.Size([30, 256, 28, 28])

### 辅助分类器
![picture](./AuxClf.jpg "picture")

In [10]:
class AuxClf(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.feature_ = nn.Sequential(nn.AvgPool2d(5, stride=3),
                                      BasicCon2d(in_channels, 128, kernel_size=1))
        self.classifier_ = nn.Sequential(nn.Flatten(),
                                         nn.Linear(2048, 1024),
                                         nn.ReLU(inplace=True),
                                         nn.Dropout(0.7),
                                         nn.Linear(1024, num_classes))
    
    def forward(self, x):
        x = self.feature_(x)
        x = self.classifier_(x)
        return x

In [11]:
# 4a
AuxClf(512,1000)

AuxClf(
  (feature_): Sequential(
    (0): AvgPool2d(kernel_size=5, stride=3, padding=0)
    (1): BasicCon2d(
      (conv): Sequential(
        (0): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
  )
  (classifier_): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=1024, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.7, inplace=False)
    (4): Linear(in_features=1024, out_features=1000, bias=True)
  )
)

### GooLeNet

In [14]:
class GooLeNet(nn.Module):
    def __init__(self, num_classes:int=1000):
        super().__init__()
        self.conv1 = BasicCon2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.conv2 = BasicCon2d(64, 64, kernel_size=1)
        self.conv3 = BasicCon2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, (96, 128), (16, 32), 32)
        self.inception3b = Inception(256, 128, (128, 192), (32, 96), 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, (96, 208), (16, 48), 64)
        self.inception4b = Inception(512, 160, (112, 224), (24, 64), 64)
        self.inception4c = Inception(512, 128, (128, 256), (24, 64), 64)   
        self.inception4d = Inception(512, 112, (144, 288), (32, 64), 64)
        self.inception4e = Inception(528, 256, (160, 320), (32, 128), 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, (160, 320), (32, 128), 128)
        self.inception5b = Inception(832, 384, (192, 384), (48, 128), 128)
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)

        # 辅助分类器
        self.aux1 = AuxClf(512, num_classes)
        self.aux2 = AuxClf(528, num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        aux1 = self.aux1(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        aux2 = self.aux2(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)

        return x, aux1, aux2

In [13]:
data = torch.randn(20, 3, 224, 224)

In [16]:
GooLeNet()

GooLeNet(
  (conv1): BasicCon2d(
    (conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicCon2d(
    (conv): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (conv3): BasicCon2d(
    (conv): Sequential(
      (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(


In [17]:
net = GooLeNet()

In [19]:
x, aux1, aux2 = net(data)

In [20]:
for i in [x, aux1, aux2]:
    print(i.shape)

torch.Size([20, 1000])
torch.Size([20, 1000])
torch.Size([20, 1000])


### 模型参数信息
![picture](./Inception%20Architecture.png "picture")

In [23]:
summary(net, (20, 3, 224, 224), device="cpu") # depth=1, 

Layer (type:depth-idx)                        Output Shape              Param #
GooLeNet                                      [20, 1000]                --
├─BasicCon2d: 1-1                             [20, 64, 112, 112]        --
│    └─Sequential: 2-1                        [20, 64, 112, 112]        --
│    │    └─Conv2d: 3-1                       [20, 64, 112, 112]        9,408
│    │    └─BatchNorm2d: 3-2                  [20, 64, 112, 112]        128
│    │    └─ReLU: 3-3                         [20, 64, 112, 112]        --
├─MaxPool2d: 1-2                              [20, 64, 56, 56]          --
├─BasicCon2d: 1-3                             [20, 64, 56, 56]          --
│    └─Sequential: 2-2                        [20, 64, 56, 56]          --
│    │    └─Conv2d: 3-4                       [20, 64, 56, 56]          4,096
│    │    └─BatchNorm2d: 3-5                  [20, 64, 56, 56]          128
│    │    └─ReLU: 3-6                         [20, 64, 56, 56]          --
├─BasicCon2d